In [1]:
import nltk
import numpy as np
import pandas as pd
from gensim.models import LdaModel
import gensim
from gensim.corpora import Dictionary, MmCorpus
import gensim.corpora as corpora
import glob

import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from wordcloud import WordCloud

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\e11925939\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('../../data/feature/clusters/0_cluster.csv')
df.head()

,ID_GodotObject,cluster_value,merged_text
0,2000116643454,0,﻿20 Normalisierung lange zurück Land Locker co...
1,2000116807446,0,﻿20 Neuerung erster Schritt Normalität ab Dien...
2,2000117799374,0,﻿20_Coronavirus zehn -quadratmeter-regel Hande...
3,2000119926812,0,﻿20_Coronavirus Regierung verschärfen corona-m...
4,2000119964770,0,﻿20 Verordnung neu corona-maßnahmen ab Montag ...


In [3]:
# Merge all text per cluster
# Find all CSV files in the current directory
filenames = glob.glob('../../data/feature/clusters/*.csv')

# Create a list to store the dataframes
df_clusters = pd.DataFrame()
# Define a function that concatenates the text from a row
def concat_text(row):
    return ' '.join(row)

# Loop over the CSV files
for filename in filenames:
    # Read the file into a dataframe
    
    df = pd.read_csv(filename)
    l = ''.join((df['merged_text'].to_numpy()))
    c = df['cluster_value'][0]
    df_clusters = df_clusters.append({'cluster' : c, 'text' : l}, ignore_index = True)
    
df_clusters.sort_values(by=['cluster'], inplace=True)
df_clusters.reset_index(drop=True, inplace=True)
df_clusters.head()

C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_74640\2095982231.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_clusters = df_clusters.append({'cluster' : c, 'text' : l}, ignore_index = True)
C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_74640\2095982231.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_clusters = df_clusters.append({'cluster' : c, 'text' : l}, ignore_index = True)
C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_74640\2095982231.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_clusters = df_clusters.append({'cluster' : c, 'text' : l}, ignore_index = True)
C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_74640\2095982231.py:18: FutureWarning: The frame.append method is deprecat

,cluster,text
0,0,﻿20 Normalisierung lange zurück Land Locker co...
1,1,﻿20_Coronavirus schrittweise Einführung Masken...
2,2,﻿20ab Freitag Regierung kündigen Maskenpflicht...
3,3,﻿20_Coronavirus Sonderbeauftragter Clemens Aue...
4,4,﻿20kein Ende Sicht Regierung setzen Maske bei ...


It is generally not necessary to remove duplicate words from a text before calculating the term frequency-inverse document frequency (TF-IDF) scores for the words. The TF-IDF score of a word takes into account both the frequency of the word in the document (term frequency, or TF) and the inverse of the frequency of the word in the collection (inverse document frequency, or IDF).

For example, consider the following two documents:

Document 1: "the cat sat on the mat"
Document 2: "the cat sat on the mat"

The term frequency of "the" in both documents is 2, because it appears twice in each document. The inverse document frequency of "the" in this collection is 1, because it appears in both documents. Therefore, the TF-IDF score of "the" in both documents is 2 * 1 = 2.

On the other hand, the term frequency of "cat" in both documents is 1, because it appears once in each document. The inverse document frequency of "cat" in this collection is 1, because it appears in both documents. Therefore, the TF-IDF score of "cat" in both documents is 1 * 1 = 1.

As you can see, the TF-IDF scores of the words in the documents are the same, regardless of whether the words are duplicated or not. Therefore, it is not necessary to remove duplicate words from a text before calculating the TF-IDF scores.

In [4]:
#TF IDS Score for labeling the clusters
# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(df_clusters['text'])

# Convert the sparse matrix to a dense array
dense_array = tfidf_vectorizer_vectors.toarray()

# Get the feature names from the TfidfVectorizer object
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a dataframe from the dense array
df = pd.DataFrame(dense_array, columns=feature_names).T.reset_index()


df.head()



,index,0,1,2,3,4,5,6,7
0,0000,0.009875,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0
1,006121,0.009875,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0
2,006721,0.009875,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0
3,024,0.000000,0.0,0.0,0.000000,0.0,0.00898,0.0,0.0
4,033,0.000000,0.0,0.0,0.008308,0.0,0.00000,0.0,0.0


In [5]:
dfs = []

for col in df:
    if col == 'index':
        continue;
    else:
        dfcol = df[['index',col]].copy().sort_values(by=col, ascending=False).reset_index(drop=True)
        dfcol.columns = ['words_'+str(col), 'tfidf_'+str(col)]
        wordcloud = dfcol.set_index('words_'+str(col))
        dfs.append(dfcol)


#print(len(dfs))

In [6]:
dfs[2].head()

,words_2,tfidf_2
0,werden,0.256893
1,neu,0.191434
2,ab,0.171262
3,montag,0.164086
4,corona,0.159029


In [7]:
result = pd.concat(dfs,axis=1)
result.head()

,words_0,tfidf_0,words_1,tfidf_1,words_2,tfidf_2,words_3,tfidf_3,words_4,tfidf_4,words_5,tfidf_5,words_6,tfidf_6,words_7,tfidf_7
0,gelten,0.335028,corona,0.229602,werden,0.256893,corona,0.275379,sein,0.287152,corona,0.200831,corona,0.198921,demo,0.415968
1,ab,0.283951,vfgh,0.199581,neu,0.191434,werden,0.209023,sagen,0.188398,mehr,0.186486,gelten,0.189925,werden,0.259725
2,maskenpflicht,0.236626,griechenland,0.183981,ab,0.171262,österreich,0.195752,maske,0.160487,österreich,0.157796,bid,0.155661,polizei,0.248024
3,dürfen,0.193999,maskenpflicht,0.146945,montag,0.164086,wien,0.165891,corona,0.152940,ab,0.154210,maskenpflicht,0.153335,anzeigen,0.220942
4,person,0.157750,covid,0.110209,corona,0.159029,schule,0.139349,geben,0.143576,wien,0.154210,ab,0.145047,wien,0.211881


In [8]:
result.to_csv('../../data/feature/tfids_cluster.csv', encoding='utf-8', index=False)

In [9]:
## WORDCLOUDS
#-- 5 Topics
# 0 = Regel, Schule, Impfung, Schutz
# 1 = Haimbuchner, Griechenland, Gesetzwidrig, FPÖ, Klage, angefochten
# 2 = Polizei, Demo, Neuinfektionen
# 3 = Maßnahmen verschärfen
# 4 = Biden, USA, ablenkend

#-- 7 topics

# 0 = Maskenpflicht, Regel, Österreich weit, FFP2
# 1 = Haimbuchner, Klage, Geldstrafe
# 2 = Schule, Maskenpflicht, Kinder
# 3 = Coronavirus aktuelle Zahlen, Hinweis, 0800 555
# 4 = Polizei, Demonstrationen
# 5 = Verfassungsgerichtshof, Griechenland, Delta Variante
# 6 = Corona Ampel Gelb, Veranstaltungen, Mückstein, regierung

# ------------> new Clustering Method

# Top 3 Groups
# 0 = Maskenpflicht Verordnungen, geltende Regeln
# 1 = Demonstrationen, Polizei, Anzeigen, ÖVP
# 2 = FPÖ, Haimbuchner, Verfassungsgerichthof, Schule, Delta Variante

#Top 8 groups
# 0 = Veranstaltungen, Handel, Wirtschaft
# 1 = Verfassungsgerichthof, Impfung, Verhältnismäß, Unverhältnismäßig
# 2 = Schramböck, Quarantäne, Grenzkontrollen und Reise
# 3 = Schule, Haimbuchner, FPÖ, Demonstration
# 4 = Mückstein, ÖVP, Skifahren, Maßnahmen
# 5 = Neuinfektionen
# 6 = Impfpflicht, ungeimpfte, geimpfte
# 7 = Demonstration, Polizei, Anzeigen, Festnahmen, Staatsgewalt

#Merged clusters
# 0 -> (0,2)
# 1 -> (1,6)
# 2 -> (3,7)
# 3 -> (4,5)